# Double Neural network for Rain estimation

In [1]:
import tensorflow as tf
import scipy.io as si
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from scipy.stats.stats import pearsonr

In [3]:
# the data file path
path =  '.'
radar_guage_path = path + '/radar-guage-2009.mat'
radar_trmm_path = path + '/radar-trmm-2009.mat'

Some functions will use in the data preprocess

In [4]:
class normalizer:
    def __init__(self, m = 0.0, s = 1.0):
        self.mean = m
        self.std = s
    def fit(self, st_data):
        '''Only works for the 2D data, the 1st dimension is the number of the samples'''
        self.mean = st_data.mean(axis = 0)
        self.std = st_data.std(axis = 0)
        print(self.mean, self.std)
    def transform(self, data):
        return (data - self.mean)/self.std

In [5]:
def generate_random_index(size):
    length = int(size * 9/10)
    random_arr = np.array(range(size))
    np.random.shuffle(random_arr)
    trainIndices = random_arr[:length]
    valIndices = random_arr[length:]
    print(trainIndices.shape,valIndices.shape)
    return trainIndices,valIndices

## NN1
features : radar reflectivity profile

label : rain rate (gauge)

NN1 : reflectivity (4 inputs) -> rain estimation (1 output)

### radar-gauge data preprocessing 

In [6]:
tmp = si.loadmat(radar_guage_path)
gauge = tmp['Yg']
radar_reflectivity = np.transpose(tmp['xv'])

In [7]:
# simple partition to train and validate
size = gauge.shape[0]
length = int(size * 9/10)
random_arr = np.array(range(size))
np.random.shuffle(random_arr)
trainIndices = random_arr[:length]
valIndices = random_arr[length:]
print(trainIndices.shape,valIndices.shape)


(92772,) (10308,)


In [8]:
validate_data = radar_reflectivity[valIndices]
validate_label = gauge[valIndices]
train_data = radar_reflectivity[trainIndices]
train_label = gauge[trainIndices]

In [9]:
radar_nor = normalizer()
radar_nor.fit(train_data)
train_data_n = radar_nor.transform(train_data)
validate_data_n = radar_nor.transform(validate_data)


[ 31.2263487   31.31230817  31.73983404  30.89607874] [ 7.77213029  7.23646837  6.32124218  5.82685087]


In [10]:
print(train_data_n.shape, train_label.shape)

(92772, 4) (92772, 1)


In [11]:
print('normalized radar reflectivity (4 inputs): ')
print(train_data_n[:5])
print('rain rate from gauage (1 output): ')
print(train_label[:5])

normalized radar reflectivity (4 inputs): 
[[-0.38310916 -0.45271033 -0.35754641  0.04698623]
 [ 1.15604422  1.06698824  1.03925776  0.0536044 ]
 [-0.33395831 -0.22622708 -0.06910984 -0.2338156 ]
 [ 1.06177119  1.31057741  1.34253245  0.63830815]
 [ 0.00959663 -0.07790809  0.15492103  0.12998119]]
rain rate from gauage (1 output): 
[[  5.87008913]
 [ 20.11569968]
 [  5.11671747]
 [ 21.29256588]
 [  6.32942247]]


In [11]:
print('normalized radar reflectivity (4 inputs): ')
print(validate_data_n[:5])
print('rain rate from gauage (1 output): ')
print(validate_label[:5])

normalized radar reflectivity (4 inputs): 
[[-1.24952847 -1.40703575 -1.31211886 -1.41266253]
 [ 0.4494914   0.41691651  0.38445931  0.34758552]
 [-0.86540787 -0.92033975 -0.7042505  -0.26844187]
 [-0.67648256 -0.69276216 -0.23662907 -0.00331595]
 [ 0.27844698  0.1908006   0.33590332  0.84388009]]
rain rate from gauage (1 output): 
[[ 2.02190931]
 [ 4.84542428]
 [ 5.05897411]
 [ 2.53822733]
 [ 6.09634956]]


### NN1 training

In [12]:
batch_size = 256
# write create simple linear regression model
graph1 = tf.Graph()
with graph1.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(None, 4))
    tf_train_label = tf.placeholder(tf.float32, shape=(None,1))
    tf_validate_dataset = tf.constant(validate_data_n, tf.float32)
    tf_validate_label = tf.constant(validate_label, tf.float32)
    weights = {'w1': tf.Variable(tf.truncated_normal([4, 16])),
               'w2':tf.Variable(tf.truncated_normal([16, 4])),
               'w3':tf.Variable(tf.truncated_normal([4, 1])),}
    biases = {'b1':tf.Variable(tf.zeros([16])),
              'b2':tf.Variable(tf.zeros([4])),
              'b3':tf.Variable(tf.zeros([1])),}

    def model(X, w, b):
        hidden = tf.nn.relu(tf.matmul(X, w['w1']) + b['b1'])
        hidden = tf.nn.relu(tf.matmul(hidden, w['w2']) + b['b2'])
        return tf.matmul(hidden, w['w3']) + b['b3']

    predicted_label = model(tf_train_dataset, weights, biases)
    loss = tf.reduce_mean(tf.square(predicted_label - tf_train_label))
    
    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.005
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.95, staircase=True)
    op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    train_loss = tf.reduce_mean(tf.abs(predicted_label - tf_train_label))
    validate_pre = model(tf_validate_dataset, weights, biases)
    validate_ame = tf.reduce_mean(tf.abs(validate_pre - tf_validate_label))
    validate_rmse = tf.sqrt(tf.reduce_mean(tf.square(validate_pre - tf_validate_label)))
    saver_nn1 = tf.train.Saver()

In [13]:

#train the model with stochastic gradient descent training
num_steps = 15000

with tf.Session(graph = graph1) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_data_n[offset:(offset + batch_size), :]
        #print(batch_data.shape)
        batch_labels = train_label[offset:(offset + batch_size)]
        feed_dict = {tf_train_dataset : batch_data, tf_train_label : batch_labels}
        #session.run(predicted_label, feed_dict=feed_dict)
        _, l, _, r = session.run([predicted_label, loss, op, learning_rate], feed_dict=feed_dict)

        if (step % 1000 == 0):
            print('Loss at step %d: LR %f MSE %f MAE %f' % (step, r, l, train_loss.eval(feed_dict=feed_dict)))
            print("AME = %f, RMSE = %f"%(validate_ame.eval(), validate_rmse.eval()))
            pre = validate_pre.eval()
    
            
   
    save_path = saver_nn1.save(session, "NN1.ckpt")
    print(save_path)
    

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Loss at step 0: LR 0.005000 MSE 164.596848 MAE 6.917606
AME = 6.219616, RMSE = 8.790132
Loss at step 1000: LR 0.004512 MSE 40.457142 MAE 4.196101
AME = 4.353083, RMSE = 6.286668
Loss at step 2000: LR 0.004073 MSE 41.066116 MAE 4.434376
AME = 4.340227, RMSE = 6.206257
Loss at step 3000: LR 0.003675 MSE 35.310478 MAE 4.210016
AME = 4.343867, RMSE = 6.183077
Loss at step 4000: LR 0.003317 MSE 45.065392 MAE 4.434695
AME = 4.309186, RMSE = 6.177742
Loss at step 5000: LR 0.002994 MSE 36.186493 MAE 4.216193
AME = 4.316359, RMSE = 6.183685
Loss at step 6000: LR 0.002702 MSE 36.643585 MAE 4.563245
AME = 4.390855, RMSE = 6.182939
Loss at step 7000: LR 0.002438 MSE 47.621735 MAE 4.695786
AME = 4.324302, RMSE = 6.174005
Loss at step 8000: LR 0.002201 MSE 52.541786 MAE 5.067957
AME = 4.289314, RMSE = 6.160225
Loss at step 9000: LR 0.001986 MSE 35.454647 MAE 4.109489
AME = 4.339703, RMSE = 6.168905
Loss at step 100

In [15]:
!ls

checkpoint		      NN1.ckpt.index	    radar-trmm-2009.mat
NN1&2.ipynb		      NN1.ckpt.meta
NN1.ckpt.data-00000-of-00001  radar-guage-2009.mat


## NN2
features : trmm reflectivity profile

label : rain rate (radar)

NN2 : trmm reflectivity (4 inputs) -> rain estimation (1 output)

### radar-trmm data preprocessing

In [14]:
tmp = si.loadmat(radar_trmm_path)
radar = np.transpose(tmp['radar'])
trmm = np.transpose(tmp['trmm'])

In [15]:
radar.shape, trmm.shape

((7185, 4), (7185, 4))

In [16]:
radar_n = radar_nor.transform(radar)


In [17]:
with tf.Session(graph = graph1) as session:
    saver_nn1.restore(session, "./NN1.ckpt")
    feed_dict = {tf_train_dataset : radar_n}
    radar_rr = predicted_label.eval(feed_dict = feed_dict)#

In [18]:
# simple partition to train and validate
size = radar_rr.shape[0]
Ind1, Ind2 = generate_random_index(size)


(6466,) (719,)


In [19]:
validate_data_nn2 = trmm[Ind2]
validate_label_nn2 = radar_rr[Ind2]
train_data_nn2 = trmm[Ind1]
train_label_nn2 = radar_rr[Ind1]

In [20]:
trmm_nor = normalizer()
trmm_nor.fit(train_data_nn2)
train_data_n2 = trmm_nor.transform(train_data_nn2)
validate_data_n2 = trmm_nor.transform(validate_data_nn2)


[ 31.13711458  31.29384683  31.7139658   29.82048033] [ 6.67473408  6.6381645   6.39630846  6.23208654]


### NN2 training

In [21]:
batch_size = 256
# write create simple linear regression model
graph2 = tf.Graph()
with graph2.as_default():
    tf_train_dataset2 = tf.placeholder(tf.float32, shape=(None, 4))
    tf_train_label2 = tf.placeholder(tf.float32, shape=(None,1))
    tf_validate_dataset2 = tf.constant(validate_data_n2, tf.float32)
    tf_validate_label2 = tf.constant(validate_label_nn2, tf.float32)
    weights2 = {'w1': tf.Variable(tf.truncated_normal([4, 16])),
               'w2':tf.Variable(tf.truncated_normal([16, 4])),
               'w3':tf.Variable(tf.truncated_normal([4, 1])),}
    biases2 = {'b1':tf.Variable(tf.zeros([16])),
              'b2':tf.Variable(tf.zeros([4])),
              'b3':tf.Variable(tf.zeros([1])),}

    def model(X, w, b):
        hidden = tf.nn.relu(tf.matmul(X, w['w1']) + b['b1'])
        hidden = tf.nn.relu(tf.matmul(hidden, w['w2']) + b['b2'])
        return tf.matmul(hidden, w['w3']) + b['b3']

    predicted_label2 = model(tf_train_dataset2, weights2, biases2)
    loss2 = tf.reduce_mean(tf.square(predicted_label2 - tf_train_label2))
    
    # Learning rate decay
    global_step2 = tf.Variable(0)
    starter_learning_rate2 = 0.005
    learning_rate2 = tf.train.exponential_decay(starter_learning_rate2, global_step2, 500, 0.95, staircase=True)
    op2 = tf.train.GradientDescentOptimizer(learning_rate2).minimize(loss2, global_step = global_step2)
    train_loss2 = tf.reduce_mean(tf.abs(predicted_label2 - tf_train_label2))
    validate_pre2 = model(tf_validate_dataset2, weights2, biases2)
    validate_ame2 = tf.reduce_mean(tf.abs(validate_pre2 - tf_validate_label2))
    validate_rmse2 = tf.sqrt(tf.reduce_mean(tf.square(validate_pre2 - tf_validate_label2)))
    saver_nn2 = tf.train.Saver()

In [22]:

#train the model with stochastic gradient descent training
num_steps = 15000

with tf.Session(graph = graph2) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label_nn2.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_data_n2[offset:(offset + batch_size), :]
        #print(batch_data.shape)
        batch_labels = train_label_nn2[offset:(offset + batch_size)]
        feed_dict = {tf_train_dataset2 : batch_data, tf_train_label2 : batch_labels}
        #session.run(predicted_label, feed_dict=feed_dict)
        _, l, _, r = session.run([predicted_label2, loss2, op2, learning_rate2], feed_dict=feed_dict)

        if (step % 1000 == 0):
            print('Loss at step %d: LR %f MSE %f MAE %f' % (step, r, l, train_loss2.eval(feed_dict=feed_dict)))
            print("AME = %f, RMSE = %f"%(validate_ame2.eval(), validate_rmse2.eval()))
            pre = validate_pre2.eval()
            
    
            
   
    save_path = saver_nn2.save(session, "NN2.ckpt")
    print(save_path)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Loss at step 0: LR 0.005000 MSE 143.426468 MAE 5.950020
AME = 6.141518, RMSE = 7.322823
Loss at step 1000: LR 0.004512 MSE 27.223122 MAE 2.907427
AME = 2.847070, RMSE = 5.101861
Loss at step 2000: LR 0.004073 MSE 25.671253 MAE 2.917471
AME = 3.036306, RMSE = 5.007925
Loss at step 3000: LR 0.003675 MSE 29.895626 MAE 3.418831
AME = 2.804297, RMSE = 4.897387
Loss at step 4000: LR 0.003317 MSE 30.132458 MAE 3.245851
AME = 2.863469, RMSE = 4.892378
Loss at step 5000: LR 0.002994 MSE 26.241253 MAE 3.039981
AME = 2.827627, RMSE = 4.889169
Loss at step 6000: LR 0.002702 MSE 29.613354 MAE 3.206232
AME = 2.825691, RMSE = 4.889077
Loss at step 7000: LR 0.002438 MSE 23.706402 MAE 2.838670
AME = 2.895335, RMSE = 4.918851
Loss at step 8000: LR 0.002201 MSE 23.810804 MAE 2.917967
AME = 2.884224, RMSE = 4.901216
Loss at step 9000: LR 0.001986 MSE 19.174904 MAE 2.775915
AME = 2.804356, RMSE = 4.868639
Loss at step 100

In [23]:
!ls

checkpoint		      NN2.ckpt.data-00000-of-00001
NN1&2.ipynb		      NN2.ckpt.index
NN1.ckpt.data-00000-of-00001  NN2.ckpt.meta
NN1.ckpt.index		      radar-guage-2009.mat
NN1.ckpt.meta		      radar-trmm-2009.mat
